## Detector Optimization with Machine Learning

#### Surya Dutta

### Motivation

Currently, the thermal models for CUORE can take in input parameters (conductances, capacitances, working temperature for heat sink, etc), run them through a stepwise differential equation solver (Runge-Kutta), and output a pulse. This **generative** model allows us to understand pulse shape characteristics and observe how changes in the parameters affects the pulse shape. However, it is impossible to get back the initial parameters we care about for detector optimization

Let $X$ be the vector of initial inputs to our thermal model differential equations, and $Y$ be a vector of pulse characteristics to the output pulse. Right now, we can generate $Y$ given $X$, but not the other way around.

Our hope for detector optimization is to understand what values in $X$ give the best results for $Y$. This can be done by finding the **joint likelihood function** for $X$ and $Y$, given by the following equation:

$$
\prod p(X_i,Y_i) = \prod p(Y_i | X_i) \prod p(X_i)
$$

The first term of this expansion, $p(Y_i,X_i)$ can be calculated with our thermal modelling. However, the second term, $p(X_i)$, is slightly harder. This is the probability that a certain set of parameters will take place in our detector - it can be approximated to some degree using the CUORE background models, but not too well (I think...)

Let's rewrite the equation above, but change up the terms slightly:

$$
\prod p(X_i,Y_i) = \prod p(X_i | Y_i) \prod p(Y_i)
$$

Here, the second term **is** calculatable by looking at our data directly, as it is just the probability that we will see some kind of pulse. However, the first term is now problematic as it is a calculation of the parameters given a pulse. This is where **machine learning** comes in. 

By modeling the relation between these output pulse and the intial inputs, we can create some parametric model that approximates $p(X_i | Y_i)$, and therefore gives us the joint distribution needed to optimize our detector. 

Calculating this term will also immensely help evaluate our dataset, and identify parameters that caused a pulse to occur (regression task). Additionally, calculating this term would also allow us to **classify the location of energy deposition**, and therefore make the IsSignal pulse shape analysis cut more accurate (or at least, validate existing algorithms).

### Methodology

So how do we actually calculate $p(X_i, Y_i)$? 

The most straightforward method of doing this is by running the thermal model a large number of times with varying parameters, construct a dataset of (X,Y), and start running complex machine learning algorithms to get this model. However, this will require a lot of computing power to generate a data set that is good enough to get a reasonable model. 

We can selectively choose our parameters, and work our way up to a decent data set using forward stepsize selection and cross-validation. However, since we don't have generated data to work with already, we would need to rely on heuristics.

A better approach would be to start with some collected data, use known initial parameters (like working temperature of the crystal), generate a smaller test set of pulses using this data, and try to model the initial conditions from this set. This may give us insight into how to select our parameters, and would also serve as validation for the methodology. 

On the machine learning side, for whatever model we use, it is important that we include interaction variables to capture the dependences of the various parts of the model (GAMs are not viable). A neural net with a few hidden layers would probably do the trick, but for simplicity sake, we can start with a simple classification response (is it a Signal, Pulser, Heater?) and a logistic regression model.
